In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

import moscot.plotting as mpl
from moscot.problems.cross_modality import TranslationProblem
from moscot.problems.generic import GWProblem

import scanpy as sc
import anndata as ad
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing as pp

import time
from tqdm import tqdm
import seaborn as sns
import scib_metrics
import scib
import helper_functions as fcts

warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", DeprecationWarning)

In [19]:
results = []
for i in tqdm(['s1d1', 's1d2', 's1d3', 's2d1', 's2d4', 's2d5', 's3d10', 's3d3', 's3d6', 's3d7', 's4d1', 's4d8', 's4d9']):
    atac = sc.read('data/'+i+'/atac-emb_gc.h5ad')
    rna = sc.read('data/'+i+'/rna-emb_gc.h5ad')
    rna.obsm['X_pca_norm'] = pp.normalize(rna.obsm['X_pca'])
    atac.obsm['X_lsi_norm'] = pp.normalize(atac.obsm['X_lsi_red']) 
    for eps in tqdm([1e-3,.5e-2,1e-2,.5e-1,1e-1, .5]):
        tp = TranslationProblem(adata_src=rna, adata_tgt=atac)
        tp = tp.prepare(
            src_attr="X_pca_norm",
            tgt_attr="X_lsi_norm"
        )
        tp = tp.solve(alpha=1.0, epsilon=eps)
        conv = tp[('src', 'tgt')].solution.converged
        T = tp[('src', 'tgt')].solution.transport_matrix
        translated_norm = T.dot(atac.obsm['X_lsi_norm'])/ np.sum(T,axis=1)[:,None]
        adata = sc.concat([rna, atac], join = 'outer', label='batch', index_unique = '-', keys=["RNA (translated)", "ATAC"])
        adata.obsm["X_scot"] = np.concatenate((translated_norm, atac.obsm['X_lsi_norm']), axis=0)

        # Evaluation metrics
        f = fcts.foscttm(translated_norm, atac.obsm['X_lsi_norm'])
        lta = fcts.transfer_accuracy(translated_norm, atac.obsm['X_lsi_norm'], atac.obs['cell_type'], rna.obs['cell_type'])
        alg_score = fcts.alignment_score(np.array(translated_norm), np.array(atac.obsm['X_lsi_norm']))
        rmse = fcts.calculate_rmse(translated_norm, atac.obsm['X_lsi_norm'])
        
        # Bio-conservation metrics
        sc.pp.neighbors(adata, use_rep='X_scot')
        scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)
        nmi = scib.me.nmi(adata, cluster_key='cluster_key', label_key='cell_type')
        asw_label = scib.me.silhouette(adata, label_key='cell_type', embed='X_scot')
        ari = scib.me.ari(adata, cluster_key='cluster_key', label_key='cell_type')

        # Batch correction metrics
        asw_batch = scib.me.silhouette_batch(adata,batch_key='batch',label_key='cell_type',embed='X_scot',verbose=False)
        graph_connectivity = scib.me.graph_connectivity(adata, label_key='cell_type')

        results.append([i, eps, conv, f, lta, alg_score, rmse, nmi, asw_label, ari, asw_batch, graph_connectivity])

  0%|          | 0/6 [00:00<?, ?it/s]

INFO     Solving `1` problems                                                                                      
INFO     Solving problem OTProblem[stage='prepared', shape=(6224, 6224)].                                          


/dss/dsshome1/01/di93bur/miniconda3/envs/moscot_env_2/lib/python3.9/site-packages/jax/_src/deprecations.py:51: DeprecationWarning: jax.numpy.DeviceArray is deprecated. Use jax.Array.
  warnings.warn(message, DeprecationWarning)
/tmp/ipykernel_2231583/654217373.py:28: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)

 17%|█▋        | 1/6 [01:20<06:44, 80.96s/it]

INFO     Solving `1` problems                                                                                      
INFO     Solving problem OTProblem[stage='prepared', shape=(6224, 6224)].                                          


/dss/dsshome1/01/di93bur/miniconda3/envs/moscot_env_2/lib/python3.9/site-packages/jax/_src/deprecations.py:51: DeprecationWarning: jax.numpy.DeviceArray is deprecated. Use jax.Array.
  warnings.warn(message, DeprecationWarning)
/tmp/ipykernel_2231583/654217373.py:28: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)

 33%|███▎      | 2/6 [02:23<04:41, 70.28s/it]

...

/dss/dsshome1/01/di93bur/miniconda3/envs/moscot_env_2/lib/python3.9/site-packages/jax/_src/deprecations.py:51: DeprecationWarning: jax.numpy.DeviceArray is deprecated. Use jax.Array.
  warnings.warn(message, DeprecationWarning)
/tmp/ipykernel_2231583/654217373.py:28: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)

 67%|██████▋   | 4/6 [02:05<01:00, 30.32s/it]

INFO     Solving `1` problems                                                                                      
INFO     Solving problem OTProblem[stage='prepared', shape=(4325, 4325)].                                          


/dss/dsshome1/01/di93bur/miniconda3/envs/moscot_env_2/lib/python3.9/site-packages/jax/_src/deprecations.py:51: DeprecationWarning: jax.numpy.DeviceArray is deprecated. Use jax.Array.
  warnings.warn(message, DeprecationWarning)
/tmp/ipykernel_2231583/654217373.py:28: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)

 83%|████████▎ | 5/6 [02:35<00:30, 30.23s/it]

INFO     Solving `1` problems                                                                                      
INFO     Solving problem OTProblem[stage='prepared', shape=(4325, 4325)].                                          


/dss/dsshome1/01/di93bur/miniconda3/envs/moscot_env_2/lib/python3.9/site-packages/jax/_src/deprecations.py:51: DeprecationWarning: jax.numpy.DeviceArray is deprecated. Use jax.Array.
  warnings.warn(message, DeprecationWarning)
/tmp/ipykernel_2231583/654217373.py:28: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  scib.cl.opt_louvain(adata,label_key='cell_type',cluster_key='cluster_key',plot=False,inplace=True,force=True,verbose = False)

100%|██████████| 13/13 [1:04:25<00:00, 297.33s/it]


In [ ]:
df_results = pd.DataFrame(
    results,
    columns =['donor', 'epsilon', 'converged', 'foscttm', "label_transfer_accuracy", "alignment_score", "rmse", "nmi", "asw_label", "ari", "asw_batch", "graph_connectivity"]
)
df_results[['nmi', 'ari']] = df_results['nmi_ari'].apply(pd.Series)
df_results.to_csv('results_gw.csv', index = False)